<h1>Imports</h1>

In [1]:
from pathlib import Path

import numpy as np
import sklearn as slearn
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn import svm
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score
from sklearn.metrics import classification_report

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.utils.class_weight import compute_class_weight


from imblearn.combine import SMOTETomek
import math
import glob
import os

Using TensorFlow backend.


In [2]:
pd.set_option("display.max_columns", 100)
%matplotlib inline

In [3]:
RANDOM_SEED = 6

<h1>Dataset Importing</h1>

In [4]:
DATA_PATH = Path.cwd() / "../../datasets"

In [5]:
training_df = pd.read_csv(DATA_PATH / "train.csv", index_col="tripid")
training_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
tripid,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct


In [6]:
training_df = training_df.dropna(subset=['fare'])

<h1>Data Preprocessing</h1>

<h2>Data Conversions</h2>

In [ ]:
# Converting date time objects from Objects to datetime64
training_df['pickup_time'] = pd.to_datetime(training_df['pickup_time'], format="%m/%d/%Y %H:%M")
training_df['drop_time'] = pd.to_datetime(training_df['drop_time'], format="%m/%d/%Y %H:%M")

<h2>Adding Features</h2>
Adding features deemed useful from Feature Engineering

In [ ]:
def dist_from_coordinates(lat1, lon1, lat2, lon2):
  R = 6371  # Earth radius in km

  #conversion to radians
  d_lat = np.radians(lat2-lat1)
  d_lon = np.radians(lon2-lon1)

  r_lat1 = np.radians(lat1)
  r_lat2 = np.radians(lat2)

  #haversine formula
  a = np.sin(d_lat/2.) **2 + np.cos(r_lat1) * np.cos(r_lat2) * np.sin(d_lon/2.)**2

  haversine = 2 * R * np.arcsin(np.sqrt(a))

  return haversine

In [ ]:
training_df = training_df.assign(timeOfDay=pd.cut(training_df.pickup_time.dt.hour,[-1, 8, 20, 24],labels=['dawn','day', 'night']))

In [ ]:
new_column = []                    #empty column for distance
for index,row in training_df.iterrows():
  lat1 = row['pick_lat'] #first row of location.lat column here
  lon1 = row['pick_lon'] #first row of location.long column here
  lat2 = row['drop_lat'] #second row of location.lat column here
  lon2 = row['drop_lon'] #second row of location.long column here
  value = dist_from_coordinates(lat1, lon1, lat2, lon2)  #get the distance
  new_column.append(value)   #append the empty list with distance values

training_df.insert(4,"distance",new_column)

In [ ]:
durations = []
for index,row in training_df.iterrows():
  provided_duration = row['duration'] #first row of location.lat column here
  if math.isnan(provided_duration) or provided_duration <= 0 :
    time_dif = (row['drop_time'] - row['pickup_time']).seconds
    if(time_dif == 0):
        time_dif = 60
    durations.append(time_dif)
  else :  
    durations.append(provided_duration)

training_df.insert(4,"time_dif",durations)



In [ ]:
training_df['avg_speed'] = (training_df['distance'] /  ( training_df['time_dif']) * 3600 )

In [ ]:
training_df = training_df.drop(index=190167541)

In [ ]:
# Dataset after adding Columns
training_df

<h1>Model Training</h1>

<h2> Part 1 : Fare Predictor </h2>
Predict fare using features

In [ ]:
# Extract Correctly predicted fares
correct_training_df = training_df[training_df['label'] == 'correct']

In [ ]:
# Extract the useful features deemed in feature engineering
features_predictor = correct_training_df[['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'meter_waiting_till_pickup','distance','avg_speed']]
fare_prediction = correct_training_df[['fare']]

In [ ]:
features_predictor.dtypes

In [ ]:
numerical_features = features_predictor.columns[features_predictor.dtypes == "float64"].values
categorical_features = features_predictor.columns[features_predictor.dtypes == "category" ].values

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(features_predictor, fare_prediction, test_size=0.3, shuffle=True)

In [ ]:

## chain numerical preprocessing into a pipeline object
numeric_preprocessing_steps = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='median'))
])

non_numeric_preprocessing_steps = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot_encoder', OneHotEncoder(handle_unknown='ignore')),
#     ('label_encoder', LabelEncoder())
])

## create preprocessor stage of the final pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('non_numeric', non_numeric_preprocessing_steps, categorical_features),
        ('numeric', numeric_preprocessing_steps, numerical_features)
    ],
    remainder = 'drop'
)

In [ ]:
# Set of new columns
new_columns = ['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'meter_waiting_till_pickup', 'distance','avg_speed']

In [ ]:
# Transform training data columns 
preprocessed_train_features = preprocessor.fit_transform(X_train)
preprocessed_train_features_data_frame = pd.DataFrame(data=preprocessed_train_features, columns=new_columns)

In [ ]:
# Transform tvalidation data columns 
preprocessed_test_features = preprocessor.fit_transform(X_eval)
preprocessed_test_features_data_frame = pd.DataFrame(data=preprocessed_test_features, columns=new_columns)

In [ ]:
preprocessed_train_features_data_frame

In [ ]:
# Choosing the Predictor
estimator = RandomForestRegressor()

# Train the model
estimator.fit(preprocessed_train_features, y_train)

# Predict for the evaluation set
y_pred = estimator.predict(preprocessed_test_features_data_frame)
slearn.metrics.mean_squared_error(y_pred, y_eval)

In [ ]:
# training using split data done prepare features for training with full dataset !! Optional !!

required_features = training_df[['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'meter_waiting_till_pickup','distance','avg_speed']]
preprocessed_full_features = preprocessor.fit_transform(required_features)
preprocessed_full_features_data_frame = pd.DataFrame(data=preprocessed_full_features, columns=new_columns)

In [ ]:
predicted_prices = estimator.predict(preprocessed_full_features_data_frame)

In [ ]:
training_df.insert(13,'predicted_price',predicted_prices)

In [ ]:
# Training dataset after new features inserted
training_df

<h2>Part 2: Training the classifier</h2>

In [ ]:
classifier_features = training_df[['predicted_price','fare']]
classifier_label = training_df[['label']]
mapping = {'correct': 1, 'incorrect': 0}
classifier_label = classifier_label.replace({'label':mapping})

In [ ]:
np.unique(classifier_label)

In [ ]:
class_weights = compute_class_weight('balanced', np.unique(classifier_label), classifier_label['label'])

In [ ]:
class_weights

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(classifier_features, classifier_label, test_size=0.3,stratify= classifier_label, shuffle=True, )

In [ ]:
classifier = RandomForestClassifier(random_state=42, class_weight='balanced')
## Train the model
classifier.fit(X_train, y_train)

# Predict for the evaluation set
print("Training Accuracy: %.2f" % (classifier.score(X_eval, y_eval)*100), "%")
y_pred = classifier.predict(X_eval)

In [ ]:
confusion_matrix = confusion_matrix(y_eval, y_pred)
confusion_matrix

In [ ]:
print(classification_report(y_eval, y_pred))

<h1>Model Validation</h1>

<h2>Loading the test data set</h2>

In [ ]:
test_set = pd.read_csv(DATA_PATH / "test.csv", index_col="tripid")
test_set.head()

In [ ]:
test_set.isna().sum()

<h2>Feature Addition for the Test Dataset</h2>

In [ ]:
test_set['pickup_time'] = pd.to_datetime(test_set['pickup_time'], format="%m/%d/%Y %H:%M")
test_set['drop_time'] = pd.to_datetime(test_set['drop_time'], format="%m/%d/%Y %H:%M")

In [ ]:
test_set = test_set.assign(timeOfDay=pd.cut(test_set.pickup_time.dt.hour,[-1, 8, 20, 24],labels=['dawn','day', 'night']))

In [ ]:
new_column = []                    #empty column for distance
for index,row in test_set.iterrows():
  lat1 = row['pick_lat'] #first row of location.lat column here
  lon1 = row['pick_lon'] #first row of location.long column here
  lat2 = row['drop_lat'] #second row of location.lat column here
  lon2 = row['drop_lon'] #second row of location.long column here
  value = dist_from_coordinates(lat1, lon1, lat2, lon2)  #get the distance
  new_column.append(value)   #append the empty list with distance values

test_set .insert(4,"distance",new_column)

In [ ]:
durations = []
for index,row in test_set.iterrows():
  provided_duration = row['duration'] #first row of location.lat column here
  if math.isnan(provided_duration) or provided_duration <= 0 :
    time_dif = (row['drop_time'] - row['pickup_time']).seconds
    if(time_dif == 0):
        time_dif = 60
    durations.append(time_dif)
  else :  
    durations.append(provided_duration)

test_set.insert(4,"time_dif",durations)



In [ ]:
test_set['avg_speed'] = (test_set['distance'] /  ( test_set['time_dif']) * 3600 )

In [ ]:
test_set

In [ ]:
test_features = test_set[['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'meter_waiting_till_pickup','distance','avg_speed']]
preprocessed_test_features = preprocessor.fit_transform(test_features)
preprocessed_test_features_data_frame = pd.DataFrame(data=preprocessed_test_features, columns=new_columns)

<h2>Fare prediction and correctness prediction using Test Dataset </h2>

In [ ]:
test_probs = estimator.predict(preprocessed_test_features_data_frame)

In [ ]:
test_set.insert(10,'predicted_price',test_probs)

In [ ]:
classifier_test_features = test_set[['predicted_price','fare']]

In [ ]:
predicted_labels = classifier.predict(classifier_test_features)

<h2>Writing to the Submission File</h2>

In [ ]:
submission_set = pd.read_csv(DATA_PATH / "sample_submission.csv", index_col="tripid")
submission_set.head()

submission_set['prediction']= predicted_labels

In [ ]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "theNotebook = " + "'"+thename+"'";
kernel.execute(command);

In [ ]:
filename = '../../submissions/'+theNotebook+'/teamCluster_submission_{%i}.csv'
dirname = '../../submissions/'+theNotebook
fileversion = 1

if not os.path.exists(dirname):
    os.makedirs(dirname)
while glob.glob(filename.replace('{%i}',str(fileversion))) :
    fileversion+=1
submission_set.to_csv(filename.replace('{%i}',str(fileversion)), index=True)
print("Completed!")

In [ ]:
submission_set['prediction'].idxmin()

In [ ]:
submission_set['prediction'].value_counts()